## ⚙️ Import Libraries

##Connect to Drive

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/My Drive/PROVE

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/PROVE


In [ ]:
!pip install keras_cv

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import seaborn as sns
import keras_cv.layers as kcvl

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

2.17.1


In [ ]:
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
pip install --upgrade albumentations

## ⏳ Load Data

## For Kaggle

In [ ]:
data = np.load('/kaggle/input/initial-dataset/training_set.npz')
X = data['images']
y = data['labels']

## For Colab

In [ ]:
data = np.load('training_set.npz')
X = data['images']
y = data['labels']

In [ ]:
X = X[:11958]
y = y[:11958]
classes = np.array(np.unique(y))
class_names = ['Basophil','Eosinophil','Erythroblast','Immature granulocytes','Lymphocyte','Monocyte','Neutrophil','Platelet']
print(classes)
print(class_names)

[0 1 2 3 4 5 6 7]
['Basophil', 'Eosinophil', 'Erythroblast', 'Immature granulocytes', 'Lymphocyte', 'Monocyte', 'Neutrophil', 'Platelet']


In [ ]:
# Ensure that y is a flat array of integers
y_flat = np.array(y).flatten()


# Get unique classes in `y_flat`
classes = np.unique(y_flat)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=classes, y=y_flat)

# Convert class weights in dictionary
class_weights_dict = dict(enumerate(class_weights))

# Print class weights
print(class_weights_dict)

{0: 1.7544014084507042, 1: 0.6853507565337001, 2: 1.3776497695852534, 3: 0.7377838104639685, 4: 1.762676886792453, 5: 1.505287009063444, 6: 0.6415236051502146, 7: 0.9097687157638467}


## 🔎 Inspect Data

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=0.25, stratify=y)

# Split test set into validation and test sets with stratification
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, random_state=seed, test_size=0.5, stratify=y_test)

# Convert targets to categorical
y_train = tfk.utils.to_categorical(y_train, num_classes=len(class_names))
y_val = tfk.utils.to_categorical(y_val, num_classes=len(class_names))
y_test = tfk.utils.to_categorical(y_test, num_classes=len(class_names))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (8968, 96, 96, 3), y_train shape: (8968, 8)
X_val shape: (1495, 96, 96, 3), y_val shape: (1495, 8)
X_test shape: (1495, 96, 96, 3), y_test shape: (1495, 8)


In [ ]:
# Definire una pipeline di trasformazioni
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(hue_shift_limit=40, sat_shift_limit=30, val_shift_limit=20, p=0.3),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.3),
    A.Defocus(radius=(1,3), alias_blur=(0.5, 0.5), p=0.3),
    A.CLAHE(clip_limit=(3, 3), tile_grid_size=(8, 8), p=0.3),
    A.Downscale(scale_range=(0.5, 0.75), interpolation_pair={'downscale': cv2.INTER_NEAREST, 'upscale': cv2.INTER_LINEAR}, p=0.3),
    A.GaussNoise(p=0.2),
    A.RandomCrop(height=70,width=70, p=0.3),
    A.RandomScale(scale_limit=0.6, p=0.2),
    A.Resize(96, 96)  # Guarantee that all images are 96x96
])

In [ ]:
# Apply transformation to the whole training dataset
X_augmented = []

for img in X_train:
    augmented = transform(image=img)
    X_augmented.append(augmented["image"])

# Convert the transformed dataset into a NumPy array
X_augmented = np.array(X_augmented)

In [ ]:
print(f"X_train shape: {X_train.shape}, X_augmented shape: {X_augmented.shape}")

X_train shape: (8968, 96, 96, 3), X_augmented shape: (8968, 96, 96, 3)


## 🛠️ General settings



In [ ]:
from keras.applications.efficientnet_v2 import EfficientNetV2M

In [ ]:
from tensorflow.keras.layers import LayerNormalization, GroupNormalization

In [ ]:
# Initialise EfficientNetV2M model with pretrained weights, for transfer learning
eff = tfk.applications.EfficientNetV2M(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet',
    include_preprocessing=True,
)
eff.trainable = False

214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


## 🛠️ Transfer Learning with Lion



In [ ]:
# Optimizer with exponential decay
optimizer = tf.keras.optimizers.Lion(1e-4)

# ReduceLROnPlateau for fine adjustments
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation accuracy
    factor=0.5,              # Halve the learning rate
    patience=5,             # Wait 10 epochs for no improvement
    min_lr=1e-6,             # Minimum learning rate
    verbose=1                # Log learning rate changes
)

In [ ]:
#DEF OF CUSTOM MODEL
def build_eff_aug(input_shape=(96, 96, 3)):

    """Build a ConvNeXtLarge-based CNN with image augmentation"""

    # Input layer
    inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')

    # EfficientNet base model
    x = eff(inputs)

    # Inception Block
    def inception_block(inputs):
        tower_1 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_1 = tfkl.Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

        tower_2 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_2 = tfkl.Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

        tower_3 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
        tower_3 = tfkl.Conv2D(64, (1, 1), activation='relu')(tower_3)

        output = tfkl.concatenate([tower_1, tower_2, tower_3], axis=-1)
        return output

    x = inception_block(x)

    # Flatten and fully connected layers with Group Normalization
    x = tfkl.Flatten()(x)
    x = tfkl.Dropout(0.5)(x)
    x = GroupNormalization(groups=8, axis=-1)(x)
    x = tfkl.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-5), name='dense_128')(x)
    x = tfkl.Dropout(0.1)(x)

    # Output layer
    outputs = tfkl.Dense(y_train.shape[-1], activation='softmax', name='dense')(x)

    # Final model
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        optimizer=optimizer,
        metrics=['accuracy']
    )
    # Return the model
    return model

In [ ]:
# Build an EffNet based custom  model for transfer learning with Lion
tl_model_Lion = build_eff_aug()

# Train the model
tl_history_Lion = tl_model_Lion.fit(
    x=X_augmented,
    y=y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_val, y_val),
    class_weight = class_weights_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True
        ),
        reduce_lr  # Add dynamic learning rate adjustment
    ]
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history_Lion['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'CNN_' + str(final_val_accuracy) + '.keras'
tl_model_Lion.save(model_filename)

# Save weights of the Transfer Learning model
tl_model_Lion.save_weights('tl.weights.h5')

# Free memory by deleting the model instance
del tl_model_Lion

Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 175s 700ms/step - accuracy: 0.2422 - loss: 2.1300 - val_accuracy: 0.7398 - val_loss: 1.1234 - learning_rate: 1.0000e-04
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 72s 87ms/step - accuracy: 0.5402 - loss: 1.4453 - val_accuracy: 0.7967 - val_loss: 1.0282 - learning_rate: 1.0000e-04
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 19s 79ms/step - accuracy: 0.6667 - loss: 1.2327 - val_accuracy: 0.8615 - val_loss: 0.9371 - learning_rate: 1.0000e-04
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - accuracy: 0.7065 - loss: 1.1474 - val_accuracy: 0.8676 - val_loss: 0.9381 - learning_rate: 1.0000e-04
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 19s 79ms/step - accuracy: 0.7378 - loss: 1.0710 - val_accuracy: 0.8756 - val_loss: 0.9291 - learning_rate: 1.0000e-04
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 21s 86ms/step - accuracy: 0.7578 - loss: 1.0438 - val_accuracy: 0.8843 - val_loss: 0.8888 - learning_rate: 1.0000e-04
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 20s 84ms/s

## 🛠️ Transfer Learning with Adam

In [ ]:
# Optimizer with exponential decay
optimizer = tf.keras.optimizers.Adam(1e-4)

# ReduceLROnPlateau for fine adjustments
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation accuracy
    factor=0.5,              # Halve the learning rate
    patience=5,             # Wait 10 epochs for no improvement
    min_lr=1e-6,             # Minimum learning rate
    verbose=1                # Log learning rate changes
)

In [ ]:
#DEF OF CUSTOM MODEL
def build_eff_aug(input_shape=(96, 96, 3)):

    """Build a ConvNeXtLarge-based CNN with image augmentation"""

    # Input layer
    inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')

    # EfficientNet base model
    x = eff(inputs)

    # Inception Block
    def inception_block(inputs):
        tower_1 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_1 = tfkl.Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

        tower_2 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_2 = tfkl.Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

        tower_3 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
        tower_3 = tfkl.Conv2D(64, (1, 1), activation='relu')(tower_3)

        output = tfkl.concatenate([tower_1, tower_2, tower_3], axis=-1)
        return output

    x = inception_block(x)

    # Flatten and fully connected layers with Group Normalization
    x = tfkl.Flatten()(x)
    x = tfkl.Dropout(0.5)(x)
    x = GroupNormalization(groups=8, axis=-1)(x)
    x = tfkl.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-5), name='dense_128')(x)
    x = tfkl.Dropout(0.1)(x)


    # Output layer
    outputs = tfkl.Dense(y_train.shape[-1], activation='softmax', name='dense')(x)

    # Final model
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        optimizer=optimizer,
        metrics=['accuracy']
    )
    # Return the model
    return model

In [ ]:
# Build an EffNet based custom  model for transfer learning with Adam
tl_model_Adam = build_eff_aug()

# Train the model
tl_history_Adam = tl_model_Adam.fit(
    x=X_augmented,
    y=y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_val, y_val),
    class_weight = class_weights_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True
        ),
        reduce_lr  # Add dynamic learning rate adjustment
    ]
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history_Adam['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'CNN_' + str(final_val_accuracy) + '.keras'
tl_model_Adam.save(model_filename)

# Save weights of the Transfer Learning model
tl_model_Adam.save_weights('tl.weights.h5')

# Free memory by deleting the model instance
del tl_model_Adam

Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 148s 535ms/step - accuracy: 0.2207 - loss: 2.2015 - val_accuracy: 0.6729 - val_loss: 1.2349 - learning_rate: 1.0000e-04
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 40s 83ms/step - accuracy: 0.4164 - loss: 1.6912 - val_accuracy: 0.7625 - val_loss: 1.0781 - learning_rate: 1.0000e-04
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.5014 - loss: 1.5090 - val_accuracy: 0.7987 - val_loss: 1.0232 - learning_rate: 1.0000e-04
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - accuracy: 0.5649 - loss: 1.3934 - val_accuracy: 0.8301 - val_loss: 1.0036 - learning_rate: 1.0000e-04
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - accuracy: 0.6133 - loss: 1.3177 - val_accuracy: 0.8348 - val_loss: 1.0146 - learning_rate: 1.0000e-04
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step - accuracy: 0.6419 - loss: 1.2570 - val_accuracy: 0.8408 - val_loss: 0.9855 - learning_rate: 1.0000e-04
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/s

## 🛠️ Transfer Learning with AdamW

In [ ]:
# Optimizer with exponential decay
optimizer = tf.keras.optimizers.AdamW(1e-4)

# ReduceLROnPlateau for fine adjustments
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation accuracy
    factor=0.5,              # Halve the learning rate
    patience=5,             # Wait 10 epochs for no improvement
    min_lr=1e-6,             # Minimum learning rate
    verbose=1                # Log learning rate changes
)

In [ ]:
#DEF OF CUSTOM MODEL
def build_eff_aug(input_shape=(96, 96, 3)):

    """Build a ConvNeXtLarge-based CNN with image augmentation"""

    # Input layer
    inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')

    # EfficientNet base model
    x = eff(inputs)

    # Inception Block
    def inception_block(inputs):
        tower_1 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_1 = tfkl.Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

        tower_2 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_2 = tfkl.Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

        tower_3 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
        tower_3 = tfkl.Conv2D(64, (1, 1), activation='relu')(tower_3)

        output = tfkl.concatenate([tower_1, tower_2, tower_3], axis=-1)
        return output

    x = inception_block(x)

    # Flatten and fully connected layers with Group Normalization
    x = tfkl.Flatten()(x)
    x = tfkl.Dropout(0.5)(x)
    x = GroupNormalization(groups=8, axis=-1)(x)
    x = tfkl.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-5), name='dense_128')(x)
    x = tfkl.Dropout(0.1)(x)

    # Output layer
    outputs = tfkl.Dense(y_train.shape[-1], activation='softmax', name='dense')(x)

    # Final model
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        optimizer=optimizer,
        metrics=['accuracy']
    )
    # Return the model
    return model

In [ ]:
# Build an EffNet based custom  model for transfer learning with AdamW
tl_model_AdamW = build_eff_aug()

# Train the model
tl_history_AdamW = tl_model_AdamW.fit(
    x=X_augmented,
    y=y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_val, y_val),
    class_weight = class_weights_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True
        ),
        reduce_lr  # Add dynamic learning rate adjustment
    ]
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history_AdamW['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'CNN_' + str(final_val_accuracy) + '.keras'
tl_model_AdamW.save(model_filename)

# Save weights of the Transfer Learning model
tl_model_AdamW.save_weights('tl.weights.h5')

# Free memory by deleting the model instance
del tl_model_AdamW

Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 151s 533ms/step - accuracy: 0.1875 - loss: 2.2970 - val_accuracy: 0.5391 - val_loss: 1.3576 - learning_rate: 1.0000e-04
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 39s 83ms/step - accuracy: 0.3978 - loss: 1.7269 - val_accuracy: 0.7117 - val_loss: 1.1222 - learning_rate: 1.0000e-04
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.5184 - loss: 1.4893 - val_accuracy: 0.7579 - val_loss: 1.0802 - learning_rate: 1.0000e-04
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 22s 91ms/step - accuracy: 0.5889 - loss: 1.3734 - val_accuracy: 0.8114 - val_loss: 1.0332 - learning_rate: 1.0000e-04
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 12s 82ms/step - accuracy: 0.6200 - loss: 1.3153 - val_accuracy: 0.8301 - val_loss: 0.9691 - learning_rate: 1.0000e-04
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.6634 - loss: 1.2464 - val_accuracy: 0.8622 - val_loss: 0.9719 - learning_rate: 1.0000e-04
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 21s 92ms/s

## 🛠️ Transfer Learning with Experimental SGD

In [ ]:
# Optimizer with exponential decay
optimizer = tf.keras.optimizers.SGD(1e-4)

# ReduceLROnPlateau for fine adjustments
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation accuracy
    factor=0.5,              # Halve the learning rate
    patience=5,             # Wait 10 epochs for no improvement
    min_lr=1e-6,             # Minimum learning rate
    verbose=1                # Log learning rate changes
)

In [ ]:
#DEF OF CUSTOM MODEL
def build_eff_aug(input_shape=(96, 96, 3)):

    """Build a ConvNeXtLarge-based CNN with image augmentation"""

    # Input layer
    inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')

    # EfficientNet base model
    x = eff(inputs)

    # Inception Block
    def inception_block(inputs):
        tower_1 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_1 = tfkl.Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

        tower_2 = tfkl.Conv2D(64, (1, 1), activation='relu')(inputs)
        tower_2 = tfkl.Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

        tower_3 = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
        tower_3 = tfkl.Conv2D(64, (1, 1), activation='relu')(tower_3)

        output = tfkl.concatenate([tower_1, tower_2, tower_3], axis=-1)
        return output

    x = inception_block(x)

    # Flatten and fully connected layers with Group Normalization
    x = tfkl.Flatten()(x)
    x = tfkl.Dropout(0.5)(x)
    x = GroupNormalization(groups=8, axis=-1)(x)
    x = tfkl.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-5), name='dense_128')(x)
    x = tfkl.Dropout(0.1)(x)

    # Output layer
    outputs = tfkl.Dense(y_train.shape[-1], activation='softmax', name='dense')(x)

    # Final model
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        optimizer=optimizer,
        metrics=['accuracy']
    )
    # Return the model
    return model

In [ ]:
# Build an EffNet based custom  model for transfer learning with SGD
tl_model_SGD = build_eff_aug()

# Train the model
tl_history_SGD = tl_model_SGD.fit(
    x=X_augmented,
    y=y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_val, y_val),
    class_weight = class_weights_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True
        ),
        reduce_lr  # Add dynamic learning rate adjustment
    ]
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history_SGD['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'CNN_' + str(final_val_accuracy) + '.keras'
tl_model_SGD.save(model_filename)

# Save weights of the Transfer Learning model
tl_model_SGD.save_weights('tl.weights.h5')

# Free memory by deleting the model instance
del tl_model_SGD

Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 148s 552ms/step - accuracy: 0.1076 - loss: 2.7068 - val_accuracy: 0.0656 - val_loss: 2.4728 - learning_rate: 1.0000e-04
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 38s 82ms/step - accuracy: 0.1169 - loss: 2.5561 - val_accuracy: 0.1204 - val_loss: 2.3005 - learning_rate: 1.0000e-04
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step - accuracy: 0.1282 - loss: 2.4451 - val_accuracy: 0.1559 - val_loss: 2.2148 - learning_rate: 1.0000e-04
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - accuracy: 0.1242 - loss: 2.4215 - val_accuracy: 0.1759 - val_loss: 2.1633 - learning_rate: 1.0000e-04
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.1369 - loss: 2.3987 - val_accuracy: 0.1926 - val_loss: 2.1167 - learning_rate: 1.0000e-04
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.1424 - loss: 2.3442 - val_accuracy: 0.1993 - val_loss: 2.0816 - learning_rate: 1.0000e-04
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/s